# Proyecto web (Semana 3)

Para este proyecto nos fue solicitado que realizaramos dos tareas:
1. Hacer uso de un API para generar un dataset.
2. Aplicar web scraping para generar un dataset.

Estas dos tareas deben resultar en los siguientes archivos:
1. Un archivo ".csv" en el cual tengamos el dataset generado via API.
2. Un archivo ".csv" en el cual tengamos el dataset generado via API, aplicando labores de limpieza y manipulación.
3. Un archivo ".csv" en el cual tengamos el dataset generado via web scraping.
4. Un archivo ".csv" en el cual tengamos el dataset generado via web scraping, aplicando labores de limpieza y manipulación.

# Ideas para el proyecto.

Considerando que los datos son la materia prima para proyectos de analitica, decidí utilizar el API de un gran sitio (Kaggle) que contiene datasets sobre diferentes temas, la gran mayoria de manera pública.

En el caso del web scraping decidí tomar una página que contiene un gran número de modelos en 3D, y obtener información acerca de cada uno de ellos.

# API 

Lo primero que realice para el uso del API, fue instalar un wrapper que ofrece el API de Kaggle.

In [ ]:
#import sys
#!{sys.executable} -m pip install mdutils kaggle 

Continue con una celda para realizar todos los imports que se vayan requiriendo a lo largo del proyecto.

In [1]:
import requests
from kaggle.api.kaggle_api_extended import KaggleApi
import time
import pandas as pd
import json
import operator
from bs4 import BeautifulSoup
import re

El wrapper del API de Kaggle realiza la autentificación con los siguientes comandos.

In [ ]:
api = KaggleApi({"username":"","key":""})
api.authenticate()

En este momento estamos autorizados para utilizar el API a partir de todos los métodos que provee el wrapper. En teoría el API de kaggle es más fácil de usar desde un shell, y su documentación (https://github.com/Kaggle/kaggle-api) esta redactada para su uso en shell. Pero es completamente factible traducir todos sus comandos al metodo incluido en el wrapper. Algunos de los comandos disponibles se enlistan a continuación:

| Comando                  | Parametros                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   | Descripción                                                                                                                                                                      |
|--------------------------|--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| dataset_list()           | sort_by: how to sort the result, see valid_dataset_sort_bys for options size: the size of the dataset, see valid_dataset_sizes for string options file_type: the format, see valid_dataset_file_types for string options license_name: string descriptor for license, see valid_dataset_license_names tag_ids: tag identifiers to filter the search search: a search term to use (default is empty string) user: username to filter the search to mine: boolean if True, group is changed to "my" to return personal page: the page to return (default is 1) | Comando para realizar búsqueda de datasets, los parámetros extra permiten ordenarlos, filtrar por tags, página que obtener, buscar datasets por usuario y otras caracteristicas. |
| dataset_view()           | :param str owner_slug: Dataset owner (required) :param str dataset_slug: Dataset name (required)                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Ver metadatos de un dataset.                                                                                                                                                     |
| dataset_metadata()       | dataset: name dataset path: its obtain with the name of the dataset.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         | Ver metadatos de un dataset.                                                                                                                                                     |
| dataset_list_files()     | dataset: the string identified of the dataset should be in format [owner]/[dataset-name]                                                                                                                                                                                                                                                                                                                                                                                                                                                                     | Lista los archivos presentes en el dataset.                                                                                                                                      |
| dataset_download_file()  | dataset: the string identified of the dataset should be in format [owner]/[dataset-name] file_name: the dataset configuration file path: if defined, download to this location force: force the download if the file already exists (default False) quiet: suppress verbose output (default is True)                                                                                                                                                                                                                                                         | Descarga un archivo presente  en un dataset.                                                                                                                                     |
| dataset_download_files() | dataset: the string identified of the dataset should be in format [owner]/[dataset-name] path: the path to download the dataset to force: force the download if the file already exists (default False) quiet: suppress verbose output (default is True) unzip: if True, unzip files upon download (default is False)                                                                                                                                                                                                                                        | Descacarga todos los archivos  de un dataset.                                                                                                                                    |
| download_file()          | response: the response to download outfile: the output file to download to quiet: suppress verbose output (default is True) chunk_size: the size of the chunk to stream                                                                                                                                                                                                                                                                                                                                                                                      | También descarga un archivo.                                                                                                                                                     |

## Extraer data.

Para mi proyecto me interesa obtener datasets que tengan relación palabras clave que yo seleccione, para esto construyo una lista con dichas palabras, en ella preferentemente hay que agregar palabras en inglés.

In [ ]:
intereses = ['currencies','currency','forex','finance','exchanges','tweets','news','fake news']

Continue realizando una búsqueda en la API con cada interes, decidí agregar una pausa entre cada solicitud a la API de un 1.5s.

In [ ]:
datasets_category = pd.DataFrame()
result_busqueda_list = []
categoria_list = []

for interes in intereses:
    time.sleep(1.5)
    response = api.dataset_list(search=interes)
    if len(response) != 0:
        result_busqueda_list.extend(response)
        categoria_list.extend(((interes+',')*len(response)).split(',')[:-1])
    
datasets_category['Dataset'] = result_busqueda_list
datasets_category['Category'] = categoria_list

In [ ]:
print('Se obtuvieron %i datasets en la busqueda sobre los interes seleccionados.' % len(datasets_category))

Revisamos si existen repeticiones en los resultados de busqueda.

In [ ]:
if len(set(datasets_category['Dataset'])) != len(datasets_category):
    print('Existen datasets repetidos')
else:
    print('No hay datasets repetidos')

Para cada uno de los datasets encontrados en la búsqueda descargaremos sus metadatos.

In [ ]:
metadata_datasets_list = []
for dataset in datasets_category['Dataset']:
    time.sleep(1)
    owner_name = str(dataset).split('/')[0]
    name = str(dataset).split('/')[1]
    metadata_datasets_list.append(api.datasets_view(owner_name,name))

In [ ]:
metadata_keys = ['id', 'ref', 'subtitle', 'tags', 'creatorName', 'creatorUrl',
                 'totalBytes', 'url', 'lastUpdated', 'downloadCount', 'isPrivate',
                 'isReviewed', 'isFeatured', 'licenseName', 'description', 'ownerName',
                 'ownerRef', 'kernelCount', 'title', 'topicCount', 'viewCount', 'voteCount',
                 'currentVersionNumber', 'files', 'versions', 'usabilityRating']

In [ ]:
metadata_df = pd.DataFrame(metadata_datasets_list)

Guardamos el dataset sin limpiar

In [ ]:
metadata_df.to_csv('dataset_api.csv')

## Selección de columnas

Seleccioné las columnas que considero útiles.

In [ ]:
useful_key_metadata = ['title','subtitle','description','lastUpdated','ref','totalBytes','url','tags','downloadCount','licenseName',
                       'kernelCount','versions','usabilityRating'] 

In [ ]:
dataset = metadata_df[useful_key_metadata]
dataset

## Identificando valores nulos.

Buscamos datos que no sean un valor.

In [ ]:
missing_values = dataset.isna().sum()
missing_values

No se encontraron valores nulos.

## Manipulación del dataset

Agregamos la columna de categoría de búsqueda con la que iniciamos.

In [ ]:
dataset['category'] = categoria_list

### Columna total bytes.

Modificamos el valor base de la columna de "Bytes" a "Mega bytes" 

In [ ]:
byte_to_gb = lambda x: x/1000000
dataset["totalBytes"] = dataset["totalBytes"].apply(byte_to_gb)
dataset = dataset.rename(columns = {"totalBytes":"totalGigaBytes"})
dataset.head(3)

## Columna Tags.

En la columna "tags" encontramos referencias a los grupos en los cuales se encuentra clasificado el dataset.
Esta columna varia de dataset a dataset. Sin embargo nos permite tener aún más grupos sobre los cuales realizar
busquedas con resultados que puedan ser de interes al usuario.

Para esta columna realizaremos una extracción de todas las etiquetas y obtenemos la frecuencia de un set para evitar repeticiones,además de que presentaremos las tres más frecuentes al usuario de manera que este pueda usarlas en una búsqueda de intereses aún mayor.

In [ ]:
suggest_interest = [element['ref'] for tag in dataset['tags'] for element in tag]
set_suggest = set(suggest_interest)
dict_freq_suggest = {k:suggest_interest.count(k) for k in set_suggest}
sorted_tups = sorted(dict_freq_suggest.items(), key=operator.itemgetter(1))
print(sorted_tups[-10:])

Estas sugerencias se pueden interpretar como los hashtag que contiene el dataset, por tanto hay que ser cuidadosos al seleccionar nuevos intereses de la lista.

Así también tenemos valores vacios para la columna etiquetas, así que sustituiremos la columna tags
por "number of tags"

In [ ]:
num_tags = lambda x: len(x)
dataset["tags"] = dataset["tags"].apply(num_tags)
dataset = dataset.rename(columns = {"tags":"numberOfTags"})
dataset.head(3)

## Columna Versions

La columna versions contiene al menos una versión para el dataset, sin embargo en caso de que tenga más solo sería
de nuestro interes la última versión y su fecha.

In [ ]:
dataset["versions"][0][0]

In [ ]:
last_version = lambda x: str(x[0]['status']) + ' version: ' +  str(x[0]['versionNumber']) + ', ' + str(x[0]['creationDate'])
dataset["versions"] = dataset["versions"].apply(last_version)
dataset = dataset.rename(columns = {"versions":"lastVersion"})
dataset.head(3)

## Análisis.

Finalmente por ahora, podemos utilizar cada una de las columnas disponibles para realizar algunos filtros con los cuales obtener información interesante.

### Mejor score de utilidad.

In [ ]:
dataset.sort_values(['usabilityRating'],ascending=False)

### Mayor tamaño

In [ ]:
dataset.sort_values(['totalGigaBytes'],ascending=False)

### El más utilizado.

In [ ]:
dataset.sort_values(['kernelCount'],ascending=False)

Guardamos el dataset limpio.

In [ ]:
dataset.to_csv('dataset_api_clean.csv')

# Web scraping.

Algo que siempre me ha gustado son los modelos 3D, estos pueden ser útiles para empresas de videojuegos, para la industria de la animación y algunos otros sectores.

La primera página que encontre con cientos de modelos de pago y gratuitos para descargar fue: https://www.turbosquid.com/

Mi objetivo será hacer web scraping a su sitio y obtener información útil para cada uno de los modelos 3D diponibles.

Generé la clase que utilizaré para realizar el scraping tomando como base el trabajo en el lab de web scraping avanzado.

In [2]:
class WebSpider:
    """
    This is the constructor class to which you can pass a bunch of parameters. 
    These parameters are stored to the class instance variables so that the
    class functions can access them later.
    
    url_pattern: the regex pattern of the web urls to scape
    pages_to_scrape: how many pages to scrape
    sleep_interval: the time interval in seconds to delay between requests. If <0, requests will not be delayed.
    content_parser: a function reference that will extract the intended info from the scraped content.
    """
    def __init__(self, url_pattern, pages_to_scrape=10, sleep_interval=-1, content_parser=None):
        self.url_pattern = url_pattern
        self.pages_to_scrape = pages_to_scrape
        self.sleep_interval = sleep_interval
        self.content_parser = content_parser
        self.output = []
    """
    Scrape the content of a single url.
    """
    def scrape_url(self, url):
        response = requests.get(url)
        if str(response) != '<Response [200]>':
            print('Error en la respuesta del servidor',response)
        elif str(response) == '<Response [408]>':
            print('Error en el limite de tiempo de respuesta del servidor')
        elif str(response) == '<Response [429]>':
            print('Error demasiadas peticiones')
        # I didn't find the SSL error but I add a 404 error catching.
        elif str(response) == '<Response [404]>':
            print('No se encontro el contenido')
        else:
            result = self.content_parser(response.content)
            self.output_results(result)
    
    """
    Export the scraped content. Right now it simply print out the results.
    But in the future you can export the results into a text file or database.
    """
    def output_results(self, r):
        self.output.extend(r)
        print('Se agregaron %i url a la lista' % (len(r)))
    """
    After the class is instantiated, call this function to start the scraping jobs.
    This function uses a FOR loop to call `scrape_url()` for each url to scrape.
    """
    def kickstart(self):
        for i in range(1, self.pages_to_scrape+1):
            if self.sleep_interval > 0:
                time.sleep(self.sleep_interval)
                self.scrape_url(self.url_pattern % i)
            else:
                self.scrape_url(self.url_pattern % i)
        return self.output

La estructura del sítio resulto ser no tan mala, sin duda una más que utiliza los div de manera impulsiva pero encontramos el url del modelo en los "div" de clase "thumbnail thumbnail-md".

La estructura para el paginado es la siguiente:

"https://www.turbosquid.com/Search/3D-Models?page_num=2&sort_column=a5&sort_order=asc"

Ya que he utilizado sus herramientas de filtrado para ordenar de menor a mayor costo, y tener los modelos gratuitos al inicio.


In [3]:
# Hacemos una prueba para la página 0
response = requests.get('https://www.turbosquid.com/Search/3D-Models?sort_column=a5&sort_order=asc')
print(response)
content = response.content
soup_prueba = BeautifulSoup(content,'html')
divs_modelos = soup_prueba.find_all('div',{'class':'thumbnail thumbnail-md'})
divs_modelos

<Response [200]>


[<div assetname="Free Base Male Anatomy" class="thumbnail thumbnail-md" id="Asset-98" thumbcount="5">
 <table cellpadding="0" cellspacing="0">
 <tr>
 <td><a href="https://www.turbosquid.com/3d-models/3d-base-anatomy-1428517"><img alt="3D base anatomy" class="" large_url="https://static.turbosquid.com/Preview/2019/07/21__10_50_25/01.jpg608BF715-5FBB-41AE-8BC7-E37020177B46DefaultHQ.jpg" src="https://static.turbosquid.com/Preview/2019/07/21__10_50_25/01.jpg608BF715-5FBB-41AE-8BC7-E37020177B46Large.jpg"/></a></td>
 </tr>
 </table>
 </div>,
 <div assetname="Minecraft Grass Block" class="thumbnail thumbnail-md" id="Asset-97" thumbcount="5">
 <table cellpadding="0" cellspacing="0">
 <tr>
 <td><a href="https://www.turbosquid.com/3d-models/3d-model-minecraft-grass-block-1428506"><img alt="3D model minecraft grass block" class="" large_url="https://static.turbosquid.com/Preview/2019/07/21__10_50_25/Grass_Block_Thumbnail_1.png9537E6EB-9C45-40A6-B467-C10A2C26401BLarge.jpg" src="https://static.turb

In [ ]:
# Para obtener el url del modelo necesitamos llegar a:
divs_modelos[0].select('a')[0]['href']

Aplicando esto a toda la página de prueba:

In [ ]:
urls_modelos = [div.select('a')[0]['href'] for div in divs_modelos]

In [ ]:
print('Tenemos %i modelos por página' % len(urls_modelos))

In [ ]:
print('Dentro de la página hay %i modelos' % (100*7668) )

Por lo tanto la función para el scraping de las url de los modelos queda de la siguiente forma:

In [4]:
def web_parser(content):
    soup_prueba = BeautifulSoup(content,'html')
    divs_modelos = soup_prueba.find_all('div',{'class':'thumbnail thumbnail-md'})
    urls_modelos = [div.select('a')[0]['href'] for div in divs_modelos]
    return urls_modelos

In [5]:
# 3D models
# https://www.turbosquid.com/Search/3D-Models?sort_column=a5&sort_order=asc
URL_PATTERN = 'https://www.turbosquid.com/Search/3D-Models?page_num=%i&sort_column=a5&sort_order=asc' # regex pattern for the urls to scrape
PAGES_TO_SCRAPE = 10 # how many webpages to scrapge
SLEEP_INTERVAL = 1

# Instantiate the IronhackSpider class
project_spider = WebSpider(URL_PATTERN,PAGES_TO_SCRAPE,SLEEP_INTERVAL, content_parser = web_parser)

# Start scraping jobs
urls_modelos_pages = project_spider.kickstart()

Se agregaron 100 url a la lista
Se agregaron 100 url a la lista
Se agregaron 100 url a la lista
Se agregaron 100 url a la lista
Se agregaron 100 url a la lista
Se agregaron 100 url a la lista
Se agregaron 100 url a la lista
Se agregaron 100 url a la lista
Se agregaron 100 url a la lista
Se agregaron 100 url a la lista


Un siguiente paso es obtener información sobre cada uno de los modelos entrando a la url obtenida y realizar scraping de nuevo, en este caso nos interesa obtener la siguiente información:

1. Nombre del modelo, div class productTitle
2. Dueño del modelo, div class productArtist
3. Precio del modelo, div class priceSection price
4. Licencia de uso, div class LicenseUses
5. Fecha de publicación
6. Formatos incluidos, tabla clase exchange
7. Categorias agregadas al modelo, accediendo al div class categorySection
8. Tags agragados al modelo, accediendo al div class tagSection
9. Descripción del modelo, accediendo al div class descriptionSection


Definimos una nueva clase de Spider.

In [10]:
class ModelsSpider:
    """
    This is the constructor class to which you can pass a bunch of parameters. 
    These parameters are stored to the class instance variables so that the
    class functions can access them later.
    
    url_pattern: the regex pattern of the web urls to scape
    pages_to_scrape: how many pages to scrape
    sleep_interval: the time interval in seconds to delay between requests. If <0, requests will not be delayed.
    content_parser: a function reference that will extract the intended info from the scraped content.
    """
    def __init__(self, urls_pages, sleep_interval=-1, content_parser=None):
        self.urls_pages = urls_pages
        self.sleep_interval = sleep_interval
        self.content_parser = content_parser
        self.output = []
    """
    Scrape the content of a single url.
    """
    def scrape_url(self, url):
        response = requests.get(url)
        if str(response) != '<Response [200]>':
            print('Error en la respuesta del servidor',response)
        elif str(response) == '<Response [408]>':
            print('Error en el limite de tiempo de respuesta del servidor')
        elif str(response) == '<Response [429]>':
            print('Error demasiadas peticiones')
        # I didn't find the SSL error but I add a 404 error catching.
        elif str(response) == '<Response [404]>':
            print('No se encontro el contenido')
        else:
            result = self.content_parser(response.content)
            self.output_results(result)
    
    """
    Export the scraped content. Right now it simply print out the results.
    But in the future you can export the results into a text file or database.
    """
    def output_results(self, r):
        self.output.append(r)
    """
    After the class is instantiated, call this function to start the scraping jobs.
    This function uses a FOR loop to call `scrape_url()` for each url to scrape.
    """
    def kickstart(self):
        for i in self.urls_pages:
            print('Scrapped')
            if self.sleep_interval > 0:
                time.sleep(self.sleep_interval)
                self.scrape_url(i)
            else:
                self.scrape_url(i)
        return self.output

Definimos un nuevo parser para extraer los datos de cada página.

In [11]:
def model_parser(content):
    check_content = lambda x: True if x != [] else False
    soup_prueba = BeautifulSoup(content,'html')
    try: 
        check_content(soup_prueba.find_all('div',{'class':'productTitle'})[0].select('h1')[0]['content'])
        nombre = soup_prueba.find_all('div',{'class':'productTitle'})[0].select('h1')[0]['content']
    except:
        nombre = ''
    try: 
        check_content(soup_prueba.find_all('div',{'class':'productArtist'})[0].text[3:])
        dueño = soup_prueba.find_all('div',{'class':'productArtist'})[0].text[3:]
    except:
        dueño = ''
    try: 
        check_content(soup_prueba.find_all('div',{'class':'priceSection price'})[0].text)
        precio = soup_prueba.find_all('div',{'class':'priceSection price'})[0].text
    except:
        precio = ''
    try: 
        check_content(soup_prueba.find_all('div',{'class':'LicenseUses'})[0].text)
        licencia = soup_prueba.find_all('div',{'class':'LicenseUses'})[0].text
    except:
        licencia = ''
    try: 
        check_content(soup_prueba.find_all('table',{'class':'SpecificationTable'})[0].select('time')[0]['datetime'])
        fecha_pub = soup_prueba.find_all('table',{'class':'SpecificationTable'})[0].select('time')[0]['datetime']
    except:
        fecha_pub = ''
    try: 
        check_content(soup_prueba.find_all('table',{'class':'exchange'})[0].text)
        formatos = soup_prueba.find_all('table',{'class':'exchange'})[0].text
    except:
        formatos = ''
    try: 
        check_content(soup_prueba.find_all('div',{'class':'FeatureGraphCategories'})[0].select('a'))
        categorias = soup_prueba.find_all('div',{'class':'FeatureGraphCategories'})[0].select('a')
        links_categorias = [categoria['href'] for categoria in categorias]
        list_categorias = [categoria.text for categoria in categorias]
    except:
        links_categorias = []
        list_categorias = []
    try: 
        check_content(soup_prueba.find_all('div',{'class':'tagSection'})[0].select('a'))
        tags = soup_prueba.find_all('div',{'class':'tagSection'})[0].select('a')
        links_tags = [tag['href'] for tag in tags]
        list_tags = [tag.text for tag in tags]
    except:
        links_tags = []
        list_tags = []
    try: 
        check_content(soup_prueba.find_all('div',{'class':'descriptionSection'})[0].select('.descriptionContentParagraph')[0].text)
        descripcion = soup_prueba.find_all('div',{'class':'descriptionSection'})[0].select('.descriptionContentParagraph')[0].text
    except:
        descripcion = ''
    row_dataset = [nombre,dueño,precio,licencia,fecha_pub,formatos,list_categorias,links_categorias,list_tags,links_tags,descripcion]
    return row_dataset

In [12]:
SLEEP_INTERVAL = 0.5

# Instantiate the IronhackSpider class
project_spider_models = ModelsSpider(urls_modelos_pages[:500],SLEEP_INTERVAL, content_parser = model_parser)

# Start scraping jobs
rows_dataset = project_spider_models.kickstart()

Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
Scrapped
S

In [13]:
rows_dataset

[['3D Free Base Male Anatomy',
  'niyoo',
  '\nFree\n',
  ' - All Extended Uses',
  '2019-07-21',
  '\n\n\nOBJ \n\n',
  ['3D Model', 'characters', 'people', 'man'],
  ['https://www.turbosquid.com/Search/3d-models',
   'https://www.turbosquid.com/3d-model/character',
   'https://www.turbosquid.com/3d-model/people',
   'https://www.turbosquid.com/3d-model/man'],
  ['characters',
   'man',
   'male',
   'human',
   'zbrush',
   'ztl',
   'obj',
   'head',
   'face',
   'body',
   'torso',
   'people',
   'free'],
  ['https://www.turbosquid.com/Search/3D-Models/characters',
   'https://www.turbosquid.com/Search/3D-Models/man',
   'https://www.turbosquid.com/Search/3D-Models/male',
   'https://www.turbosquid.com/Search/3D-Models/human',
   'https://www.turbosquid.com/Search/3D-Models/zbrush',
   'https://www.turbosquid.com/Search/3D-Models/ztl',
   'https://www.turbosquid.com/Search/3D-Models/obj',
   'https://www.turbosquid.com/Search/3D-Models/head',
   'https://www.turbosquid.com/Search/

In [14]:
dataset_scraping = pd.DataFrame(rows_dataset)
dataset_scraping.columns = [['name_model','owner','price','license','published_date','formats_available','list_categories','links_categories','list_tags','links_tags','description']]
dataset_scraping

,name_model,owner,price,license,published_date,formats_available,list_categories,links_categories,list_tags,links_tags,description
0,3D Free Base Male Anatomy,niyoo,\nFree\n,- All Extended Uses,2019-07-21,\n\n\nOBJ \n\n,"[3D Model, characters, people, man]","[https://www.turbosquid.com/Search/3d-models, ...","[characters, man, male, human, zbrush, ztl, ob...",[https://www.turbosquid.com/Search/3D-Models/c...,Free model. You can use for base anatomy/propo...
1,3D model Minecraft Grass Block,Render at Night,\nFree\n,- Editorial Uses Only,2019-07-21,\n\n\nOBJ \n\n,"[3D Model, nature, plants, grasses, ornamental...","[https://www.turbosquid.com/Search/3d-models, ...","[minecraft, grass, block, cube]",[https://www.turbosquid.com/Search/3D-Models/m...,
2,Diving Helmet 3D model,NotJerd,\nFree\n,- All Extended Uses,2019-07-20,\n\n\nOBJ \n\n,"[3D Model, sports, outdoor sports, scuba, divi...","[https://www.turbosquid.com/Search/3d-models, ...","[Helmet, Undersea]",[https://www.turbosquid.com/Search/3D-Models/h...,This is a helmet I made.CC0 License
3,3D Dominoes,Render at Night,\nFree\n,- All Extended Uses,2019-07-20,\n\n\nOBJ \n\n,"[3D Model, toys and games, games, dominos]","[https://www.turbosquid.com/Search/3d-models, ...","[domino, dominoes, black, white, tile]",[https://www.turbosquid.com/Search/3D-Models/d...,Models of 5 different dominoes.Polygons/Vertic...
4,Prototyping Polygons 3D model,HyperChromatica,\nFree\n,- All Extended Uses,2019-07-20,\n\n\nFBX 1.0\n\n,"[3D Model, symbols]","[https://www.turbosquid.com/Search/3d-models, ...","[primitives, prototyping, blender, unity, geom...",[https://www.turbosquid.com/Search/3D-Models/p...,I was starting work on a new game in unity and...
5,3D counter and bar stools,ESalem,\nFree\n,- All Extended Uses,2019-07-20,\n\n\n3D Studio \n\n\n\n\nFBX \n\n\n\n\nOBJ \n\n,"[3D Model, furnishings, cupboard, showcase]","[https://www.turbosquid.com/Search/3d-models, ...","[counter, and, bar, stools, Are, same, specifi...",[https://www.turbosquid.com/Search/3D-Models/c...,counter and bar stools Are the same specificat...
6,3D model Cabin Shop,MarkoffIN,\nFree\n,- All Extended Uses,2019-07-20,\n\n\nFBX \n\n,"[3D Model, architecture, building, commercial ...","[https://www.turbosquid.com/Search/3d-models, ...","[House, shop, store, stall, depot, trade, cabin.]",[https://www.turbosquid.com/Search/3D-Models/h...,Small low poly shop cabin.
7,3D Viper sniper rifle,Young_Wizard,\nFree\n,- All Extended Uses,2019-07-20,\n\n\nFBX \n\n\n\n\nOBJ \n\n\n\n\nOther \n\n,"[3D Model, weaponry, weapons, firearms, rifle,...","[https://www.turbosquid.com/Search/3d-models, ...","[weapon, sci-fi, low-poly, game-ready, sniper-...",[https://www.turbosquid.com/Search/3D-Models/w...,"Game ready model, low poly can used as a game ..."
8,Atom model,Abdo Ashraf,\nFree\n,- All Extended Uses,2019-07-20,,"[3D Model, science, chemistry, atom]","[https://www.turbosquid.com/Search/3d-models, ...","[atom, science, chemistry, chemical, scientifi...",[https://www.turbosquid.com/Search/3D-Models/a...,3D model for the atom. The whole project was m...
9,3D Fan,Akumax Maxime,\nFree\n,- All Extended Uses,2019-07-19,\n\n\nOBJ \n\n,"[3D Model, interior design, housewares, fan, b...","[https://www.turbosquid.com/Search/3d-models, ...","[Fan, ventilateur]",[https://www.turbosquid.com/Search/3D-Models/f...,Vintage fan


Por el momento he realizado el scraping a 10 páginas del sitio, con cien modelos, lo que representa 1000 requests. He mantenido este número por la posibilidad de que bloquen la dirección IP, sin embargo puede modificarse el parametro del primer Spider y tratar de obtener los url de todos los modelos disponibles.

In [15]:
# Guardamos en archivo csv los resultados obtenidos.
dataset_scraping.to_csv('dataset_scraping.csv',index=False)

## Limpieza de datos.

Es necesario aplicar una limpieza a los datos.

Revisamos la cantida de valores vacios

In [16]:
missing_values_scraping = dataset_scraping.isna()
missing_values_scraping = missing_values_scraping.sum()
missing_values_scraping

name_model           0
owner                0
price                0
license              0
published_date       0
formats_available    0
list_categories      0
links_categories     0
list_tags            0
links_tags           0
description          0
dtype: int64

## Columna name_model

En este caso podemos darle un formato más limpio a los nombres (capitalize) y eliminar palabras que son innecesarias.

With the name we now that all are 3D models, so we remove this word for all the names.

In [17]:
dataset_scraping.columns

MultiIndex(levels=[['description', 'formats_available', 'license', 'links_categories', 'links_tags', 'list_categories', 'list_tags', 'name_model', 'owner', 'price', 'published_date']],
           codes=[[7, 8, 9, 2, 10, 1, 5, 3, 6, 4, 0]])

Here I had a problem because I got a multiindex dataframe, so I had to apply some extra steps each time I wanted to modify a column.

In [18]:
nombres = dataset_scraping['name_model']
nombres = [nombres.loc[[i]].values[0][0] for i in range(len(nombres))]
nombres = [nombre.replace('3D ','').replace('model ','').replace('3D','').replace('model','').capitalize() for nombre in nombres]
dataset_scraping['name_model'] = nombres

## Columna price.

Primero nos damos una idea de la variedad de precios en esta columna.

In [19]:
prices = dataset_scraping['price']
prices = [prices.loc[[i]].values[0][0] for i in range(len(prices))]

Parece que dentro de los primeros 1000 modelos todos son gratuitos, por lo que solo limpiamos un poco el string.

In [20]:
dataset_scraping['price'] = [re.sub(r'\n','',price) for price in prices]
dataset_scraping

,name_model,owner,price,license,published_date,formats_available,list_categories,links_categories,list_tags,links_tags,description
0,Free base male anatomy,niyoo,Free,- All Extended Uses,2019-07-21,\n\n\nOBJ \n\n,"[3D Model, characters, people, man]","[https://www.turbosquid.com/Search/3d-models, ...","[characters, man, male, human, zbrush, ztl, ob...",[https://www.turbosquid.com/Search/3D-Models/c...,Free model. You can use for base anatomy/propo...
1,Minecraft grass block,Render at Night,Free,- Editorial Uses Only,2019-07-21,\n\n\nOBJ \n\n,"[3D Model, nature, plants, grasses, ornamental...","[https://www.turbosquid.com/Search/3d-models, ...","[minecraft, grass, block, cube]",[https://www.turbosquid.com/Search/3D-Models/m...,
2,Diving helmet,NotJerd,Free,- All Extended Uses,2019-07-20,\n\n\nOBJ \n\n,"[3D Model, sports, outdoor sports, scuba, divi...","[https://www.turbosquid.com/Search/3d-models, ...","[Helmet, Undersea]",[https://www.turbosquid.com/Search/3D-Models/h...,This is a helmet I made.CC0 License
3,Dominoes,Render at Night,Free,- All Extended Uses,2019-07-20,\n\n\nOBJ \n\n,"[3D Model, toys and games, games, dominos]","[https://www.turbosquid.com/Search/3d-models, ...","[domino, dominoes, black, white, tile]",[https://www.turbosquid.com/Search/3D-Models/d...,Models of 5 different dominoes.Polygons/Vertic...
4,Prototyping polygons,HyperChromatica,Free,- All Extended Uses,2019-07-20,\n\n\nFBX 1.0\n\n,"[3D Model, symbols]","[https://www.turbosquid.com/Search/3d-models, ...","[primitives, prototyping, blender, unity, geom...",[https://www.turbosquid.com/Search/3D-Models/p...,I was starting work on a new game in unity and...
5,Counter and bar stools,ESalem,Free,- All Extended Uses,2019-07-20,\n\n\n3D Studio \n\n\n\n\nFBX \n\n\n\n\nOBJ \n\n,"[3D Model, furnishings, cupboard, showcase]","[https://www.turbosquid.com/Search/3d-models, ...","[counter, and, bar, stools, Are, same, specifi...",[https://www.turbosquid.com/Search/3D-Models/c...,counter and bar stools Are the same specificat...
6,Cabin shop,MarkoffIN,Free,- All Extended Uses,2019-07-20,\n\n\nFBX \n\n,"[3D Model, architecture, building, commercial ...","[https://www.turbosquid.com/Search/3d-models, ...","[House, shop, store, stall, depot, trade, cabin.]",[https://www.turbosquid.com/Search/3D-Models/h...,Small low poly shop cabin.
7,Viper sniper rifle,Young_Wizard,Free,- All Extended Uses,2019-07-20,\n\n\nFBX \n\n\n\n\nOBJ \n\n\n\n\nOther \n\n,"[3D Model, weaponry, weapons, firearms, rifle,...","[https://www.turbosquid.com/Search/3d-models, ...","[weapon, sci-fi, low-poly, game-ready, sniper-...",[https://www.turbosquid.com/Search/3D-Models/w...,"Game ready model, low poly can used as a game ..."
8,Atom,Abdo Ashraf,Free,- All Extended Uses,2019-07-20,,"[3D Model, science, chemistry, atom]","[https://www.turbosquid.com/Search/3d-models, ...","[atom, science, chemistry, chemical, scientifi...",[https://www.turbosquid.com/Search/3D-Models/a...,3D model for the atom. The whole project was m...
9,Fan,Akumax Maxime,Free,- All Extended Uses,2019-07-19,\n\n\nOBJ \n\n,"[3D Model, interior design, housewares, fan, b...","[https://www.turbosquid.com/Search/3d-models, ...","[Fan, ventilateur]",[https://www.turbosquid.com/Search/3D-Models/f...,Vintage fan


## Columna formats_available

Para esta columna necesitamos un formao similar a la columna price, solo que generamos una lista de formatos.

In [21]:
formatos = dataset_scraping['formats_available']
formatos = [formatos.loc[[i]].values[0][0] for i in range(len(formatos))]

In [22]:
dataset_scraping['formats_available'] = [re.findall(r'\b\w+\s(?:\w+)?',str(formato)) for formato in formatos]
dataset_scraping

,name_model,owner,price,license,published_date,formats_available,list_categories,links_categories,list_tags,links_tags,description
0,Free base male anatomy,niyoo,Free,- All Extended Uses,2019-07-21,[OBJ ],"[3D Model, characters, people, man]","[https://www.turbosquid.com/Search/3d-models, ...","[characters, man, male, human, zbrush, ztl, ob...",[https://www.turbosquid.com/Search/3D-Models/c...,Free model. You can use for base anatomy/propo...
1,Minecraft grass block,Render at Night,Free,- Editorial Uses Only,2019-07-21,[OBJ ],"[3D Model, nature, plants, grasses, ornamental...","[https://www.turbosquid.com/Search/3d-models, ...","[minecraft, grass, block, cube]",[https://www.turbosquid.com/Search/3D-Models/m...,
2,Diving helmet,NotJerd,Free,- All Extended Uses,2019-07-20,[OBJ ],"[3D Model, sports, outdoor sports, scuba, divi...","[https://www.turbosquid.com/Search/3d-models, ...","[Helmet, Undersea]",[https://www.turbosquid.com/Search/3D-Models/h...,This is a helmet I made.CC0 License
3,Dominoes,Render at Night,Free,- All Extended Uses,2019-07-20,[OBJ ],"[3D Model, toys and games, games, dominos]","[https://www.turbosquid.com/Search/3d-models, ...","[domino, dominoes, black, white, tile]",[https://www.turbosquid.com/Search/3D-Models/d...,Models of 5 different dominoes.Polygons/Vertic...
4,Prototyping polygons,HyperChromatica,Free,- All Extended Uses,2019-07-20,"[FBX 1, 0\n]","[3D Model, symbols]","[https://www.turbosquid.com/Search/3d-models, ...","[primitives, prototyping, blender, unity, geom...",[https://www.turbosquid.com/Search/3D-Models/p...,I was starting work on a new game in unity and...
5,Counter and bar stools,ESalem,Free,- All Extended Uses,2019-07-20,"[3D Studio, FBX , OBJ ]","[3D Model, furnishings, cupboard, showcase]","[https://www.turbosquid.com/Search/3d-models, ...","[counter, and, bar, stools, Are, same, specifi...",[https://www.turbosquid.com/Search/3D-Models/c...,counter and bar stools Are the same specificat...
6,Cabin shop,MarkoffIN,Free,- All Extended Uses,2019-07-20,[FBX ],"[3D Model, architecture, building, commercial ...","[https://www.turbosquid.com/Search/3d-models, ...","[House, shop, store, stall, depot, trade, cabin.]",[https://www.turbosquid.com/Search/3D-Models/h...,Small low poly shop cabin.
7,Viper sniper rifle,Young_Wizard,Free,- All Extended Uses,2019-07-20,"[FBX , OBJ , Other ]","[3D Model, weaponry, weapons, firearms, rifle,...","[https://www.turbosquid.com/Search/3d-models, ...","[weapon, sci-fi, low-poly, game-ready, sniper-...",[https://www.turbosquid.com/Search/3D-Models/w...,"Game ready model, low poly can used as a game ..."
8,Atom,Abdo Ashraf,Free,- All Extended Uses,2019-07-20,[],"[3D Model, science, chemistry, atom]","[https://www.turbosquid.com/Search/3d-models, ...","[atom, science, chemistry, chemical, scientifi...",[https://www.turbosquid.com/Search/3D-Models/a...,3D model for the atom. The whole project was m...
9,Fan,Akumax Maxime,Free,- All Extended Uses,2019-07-19,[OBJ ],"[3D Model, interior design, housewares, fan, b...","[https://www.turbosquid.com/Search/3d-models, ...","[Fan, ventilateur]",[https://www.turbosquid.com/Search/3D-Models/f...,Vintage fan


Finalmente podemos hacer un reordenamiento de las columnas.

In [23]:
orden_columnas = ['name_model', 'description','owner','price','license','published_date',
                  'formats_available','list_categories','links_categories','list_tags','links_tags']

In [24]:
dataset_scraping = dataset_scraping[orden_columnas]
dataset_scraping

,name_model,description,owner,price,license,published_date,formats_available,list_categories,links_categories,list_tags,links_tags
0,Free base male anatomy,Free model. You can use for base anatomy/propo...,niyoo,Free,- All Extended Uses,2019-07-21,[OBJ ],"[3D Model, characters, people, man]","[https://www.turbosquid.com/Search/3d-models, ...","[characters, man, male, human, zbrush, ztl, ob...",[https://www.turbosquid.com/Search/3D-Models/c...
1,Minecraft grass block,,Render at Night,Free,- Editorial Uses Only,2019-07-21,[OBJ ],"[3D Model, nature, plants, grasses, ornamental...","[https://www.turbosquid.com/Search/3d-models, ...","[minecraft, grass, block, cube]",[https://www.turbosquid.com/Search/3D-Models/m...
2,Diving helmet,This is a helmet I made.CC0 License,NotJerd,Free,- All Extended Uses,2019-07-20,[OBJ ],"[3D Model, sports, outdoor sports, scuba, divi...","[https://www.turbosquid.com/Search/3d-models, ...","[Helmet, Undersea]",[https://www.turbosquid.com/Search/3D-Models/h...
3,Dominoes,Models of 5 different dominoes.Polygons/Vertic...,Render at Night,Free,- All Extended Uses,2019-07-20,[OBJ ],"[3D Model, toys and games, games, dominos]","[https://www.turbosquid.com/Search/3d-models, ...","[domino, dominoes, black, white, tile]",[https://www.turbosquid.com/Search/3D-Models/d...
4,Prototyping polygons,I was starting work on a new game in unity and...,HyperChromatica,Free,- All Extended Uses,2019-07-20,"[FBX 1, 0\n]","[3D Model, symbols]","[https://www.turbosquid.com/Search/3d-models, ...","[primitives, prototyping, blender, unity, geom...",[https://www.turbosquid.com/Search/3D-Models/p...
5,Counter and bar stools,counter and bar stools Are the same specificat...,ESalem,Free,- All Extended Uses,2019-07-20,"[3D Studio, FBX , OBJ ]","[3D Model, furnishings, cupboard, showcase]","[https://www.turbosquid.com/Search/3d-models, ...","[counter, and, bar, stools, Are, same, specifi...",[https://www.turbosquid.com/Search/3D-Models/c...
6,Cabin shop,Small low poly shop cabin.,MarkoffIN,Free,- All Extended Uses,2019-07-20,[FBX ],"[3D Model, architecture, building, commercial ...","[https://www.turbosquid.com/Search/3d-models, ...","[House, shop, store, stall, depot, trade, cabin.]",[https://www.turbosquid.com/Search/3D-Models/h...
7,Viper sniper rifle,"Game ready model, low poly can used as a game ...",Young_Wizard,Free,- All Extended Uses,2019-07-20,"[FBX , OBJ , Other ]","[3D Model, weaponry, weapons, firearms, rifle,...","[https://www.turbosquid.com/Search/3d-models, ...","[weapon, sci-fi, low-poly, game-ready, sniper-...",[https://www.turbosquid.com/Search/3D-Models/w...
8,Atom,3D model for the atom. The whole project was m...,Abdo Ashraf,Free,- All Extended Uses,2019-07-20,[],"[3D Model, science, chemistry, atom]","[https://www.turbosquid.com/Search/3d-models, ...","[atom, science, chemistry, chemical, scientifi...",[https://www.turbosquid.com/Search/3D-Models/a...
9,Fan,Vintage fan,Akumax Maxime,Free,- All Extended Uses,2019-07-19,[OBJ ],"[3D Model, interior design, housewares, fan, b...","[https://www.turbosquid.com/Search/3d-models, ...","[Fan, ventilateur]",[https://www.turbosquid.com/Search/3D-Models/f...


Ahora sustituimos los valores nulos y listas vacias por la cadena "Unknown"

In [25]:
dataset_scraping.fillna('No disponible')
change_to_unknown = lambda x: 'Unknown' if x == [] else x
for i in dataset_scraping.columns:
    dataset_scraping[i] = dataset_scraping[i].apply(change_to_unknown)
dataset_scraping

,name_model,description,owner,price,license,published_date,formats_available,list_categories,links_categories,list_tags,links_tags
0,Free base male anatomy,Free model. You can use for base anatomy/propo...,niyoo,Free,- All Extended Uses,2019-07-21,[OBJ ],"[3D Model, characters, people, man]","[https://www.turbosquid.com/Search/3d-models, ...","[characters, man, male, human, zbrush, ztl, ob...",[https://www.turbosquid.com/Search/3D-Models/c...
1,Minecraft grass block,,Render at Night,Free,- Editorial Uses Only,2019-07-21,[OBJ ],"[3D Model, nature, plants, grasses, ornamental...","[https://www.turbosquid.com/Search/3d-models, ...","[minecraft, grass, block, cube]",[https://www.turbosquid.com/Search/3D-Models/m...
2,Diving helmet,This is a helmet I made.CC0 License,NotJerd,Free,- All Extended Uses,2019-07-20,[OBJ ],"[3D Model, sports, outdoor sports, scuba, divi...","[https://www.turbosquid.com/Search/3d-models, ...","[Helmet, Undersea]",[https://www.turbosquid.com/Search/3D-Models/h...
3,Dominoes,Models of 5 different dominoes.Polygons/Vertic...,Render at Night,Free,- All Extended Uses,2019-07-20,[OBJ ],"[3D Model, toys and games, games, dominos]","[https://www.turbosquid.com/Search/3d-models, ...","[domino, dominoes, black, white, tile]",[https://www.turbosquid.com/Search/3D-Models/d...
4,Prototyping polygons,I was starting work on a new game in unity and...,HyperChromatica,Free,- All Extended Uses,2019-07-20,"[FBX 1, 0\n]","[3D Model, symbols]","[https://www.turbosquid.com/Search/3d-models, ...","[primitives, prototyping, blender, unity, geom...",[https://www.turbosquid.com/Search/3D-Models/p...
5,Counter and bar stools,counter and bar stools Are the same specificat...,ESalem,Free,- All Extended Uses,2019-07-20,"[3D Studio, FBX , OBJ ]","[3D Model, furnishings, cupboard, showcase]","[https://www.turbosquid.com/Search/3d-models, ...","[counter, and, bar, stools, Are, same, specifi...",[https://www.turbosquid.com/Search/3D-Models/c...
6,Cabin shop,Small low poly shop cabin.,MarkoffIN,Free,- All Extended Uses,2019-07-20,[FBX ],"[3D Model, architecture, building, commercial ...","[https://www.turbosquid.com/Search/3d-models, ...","[House, shop, store, stall, depot, trade, cabin.]",[https://www.turbosquid.com/Search/3D-Models/h...
7,Viper sniper rifle,"Game ready model, low poly can used as a game ...",Young_Wizard,Free,- All Extended Uses,2019-07-20,"[FBX , OBJ , Other ]","[3D Model, weaponry, weapons, firearms, rifle,...","[https://www.turbosquid.com/Search/3d-models, ...","[weapon, sci-fi, low-poly, game-ready, sniper-...",[https://www.turbosquid.com/Search/3D-Models/w...
8,Atom,3D model for the atom. The whole project was m...,Abdo Ashraf,Free,- All Extended Uses,2019-07-20,Unknown,"[3D Model, science, chemistry, atom]","[https://www.turbosquid.com/Search/3d-models, ...","[atom, science, chemistry, chemical, scientifi...",[https://www.turbosquid.com/Search/3D-Models/a...
9,Fan,Vintage fan,Akumax Maxime,Free,- All Extended Uses,2019-07-19,[OBJ ],"[3D Model, interior design, housewares, fan, b...","[https://www.turbosquid.com/Search/3d-models, ...","[Fan, ventilateur]",[https://www.turbosquid.com/Search/3D-Models/f...


Guardamos el dataset limpio.

In [26]:
dataset_scraping.to_csv('datase_scraping_clean.csv',index=False)

## Análisis.

Ahora podemos obtener las categorias o etiquetas más frecuentes.

In [27]:
lista_categorias = dataset_scraping['list_categories']
lista_categorias = [lista_categorias.loc[[i]].values[0][0] for i in range(len(lista_categorias))]
lista_categorias_total = [categoria for lista in lista_categorias for categoria in lista]
set_categorias = set(lista_categorias_total)
dict_freq_categories = {k:lista_categorias_total.count(k) for k in set_categorias}
freq_categories_order = sorted(dict_freq_categories.items(), key=operator.itemgetter(1))
print('Categorias más frecuentes en orden ascendente:')
print(freq_categories_order[-10:])

Categorias más frecuentes en orden ascendente:
[('housewares', 40), ('weapons', 41), ('lamp', 43), ('architecture', 50), ('weaponry', 51), ('lighting', 51), ('fixtures', 53), ('furnishings', 75), ('interior design', 107), ('3D Model', 492)]


In [28]:
lista_etiquetas = dataset_scraping['list_tags']
lista_etiquetas = [lista_etiquetas.loc[[i]].values[0][0] for i in range(len(lista_etiquetas))]
lista_etiquetas_total = [etiqueta for lista in lista_etiquetas for etiqueta in lista]
set_etiquetas = set(lista_etiquetas_total)
dict_freq_tags = {k:lista_etiquetas_total.count(k) for k in set_etiquetas}
freq_tags_order = sorted(dict_freq_tags.items(), key=operator.itemgetter(1))
print('Categorias más frecuentes en orden ascendente:')
print(freq_tags_order[-10:])

Categorias más frecuentes en orden ascendente:
[('poly', 26), ('furniture', 31), ('lowpoly', 32), ('Bedroom', 38), ('Painted-metal', 39), ('free', 39), ('game', 41), ('3d', 45), ('Contemporary-design', 52), ('Living', 61)]


Buscar los "dueños" más frecuentes.

In [29]:
dueños = dataset_scraping['owner']
dueños = [dueños.loc[[i]].values[0][0] for i in range(len(dueños))]
len(set(dueños))

292

In [30]:
dueños_dict = {k:dueños.count(k) for k in set(dueños)}
freq_dueños_order = sorted(dueños_dict.items(), key=operator.itemgetter(1))
print('Los dueños con más modelos en orden ascendente:')
print(freq_dueños_order[-10:])

Los dueños con más modelos en orden ascendente:
[('usman039', 4), ('tauan_bellintani', 5), ('w1050263', 5), ('renedominick1999', 5), ('Joshua Kolby', 5), ('desmundo', 8), ('DTG Amusements', 9), ('AdrienJ', 9), ('Brian31', 10), ('Designconnected', 63)]


La posibilidad de filtrar para encontrar información relevante sobre que dueño es el más activo con base en la fecha de publicación de sus modelos y el número de modelos es posible calcularla con base en el dataset limpio.

Ahora bien como podemos ver tambien sería interesante analizar información sobre los modelos de pago que desgrciadamente no alcanzamos a obtener debido a la inmensa cantidad de modelos que hay en la página. Por ahora se han prácticado las habilidades de uso de APIs y web scraping.